In [13]:
import pandas as pd
import numpy as np

In [14]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [15]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [16]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['Gender']=encoder.fit_transform(df['Gender'])

In [19]:
from sklearn.preprocessing import OneHotEncoder
One_hot_encoder=OneHotEncoder()
encoded=One_hot_encoder.fit_transform(df[['Geography']]).toarray()

In [21]:
encoded_df=pd.DataFrame(encoded,columns=One_hot_encoder.get_feature_names_out(['Geography']))

In [22]:
df=pd.concat([df.drop(['Geography'],axis=1),encoded_df],axis=1)

In [23]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.drop(['EstimatedSalary'],axis=1),df['EstimatedSalary'],test_size=0.20,random_state=42)

In [26]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [46]:
import pickle

with open('Label_Encoder_Gender.pkl','wb') as file:
    pickle.dump(encoder,file)

with open('One_hot_encoder_Geo.pkl','wb') as file:
    pickle.dump(One_hot_encoder,file)

with open('Standard_Scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [28]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [31]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train_scaled.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
import datetime
from tensorflow.keras.callbacks import TensorBoard
log_dir='log/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [35]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [37]:
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping_callback,tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 4s 8ms/step - loss: 72733.3438 - mae: 72733.3438 - val_loss: 66809.2188 - val_mae: 66809.2188
Epoch 2/100
250/250 [==============================] - 2s 6ms/step - loss: 65363.8203 - mae: 65363.8203 - val_loss: 61037.2266 - val_mae: 61037.2266
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 56812.1758 - mae: 56812.1758 - val_loss: 52181.8867 - val_mae: 52181.8867
Epoch 4/100
250/250 [==============================] - 1s 6ms/step - loss: 52200.2930 - mae: 52200.2930 - val_loss: 51048.4688 - val_mae: 51048.4688
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 51722.5547 - mae: 51722.5547 - val_loss: 50627.4258 - val_mae: 50627.4258
Epoch 6/100
250/250 [==============================] - 2s 6ms/step - loss: 51449.4609 - mae: 51449.4609 - val_loss: 50569.1484 - val_mae: 50569.1484
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 51366.6836 - mae: 51366.6836 

In [38]:
%load_ext tensorboard

In [43]:
%tensorboard --logdir log/fit

Reusing TensorBoard on port 6006 (pid 53132), started 0:01:14 ago. (Use '!kill 53132' to kill it.)

In [45]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f"Test MAE : {test_mae}")

63/63 [==============================] - 0s 2ms/step - loss: 50302.1797 - mae: 50302.1797
Test MAE : 50302.1796875


In [41]:
model.save('regression_model.h5')

c:\Users\KIIT0001\Desktop\ANN_Regression_Project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
